<a href="https://colab.research.google.com/github/tabesink/AIE5/blob/main/AIE5_Midterm_Finetuning_Snowflake_Arctic_Embedding_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

collections				 mg_alloy_finetuned_arctic_embed_l_ft_v2
contextual_chunks			 test_dataset.jsonl
mg_alloy_finetuned_arctic_embed_l_ft	 training_dataset.jsonl
mg_alloy_finetuned_arctic_embed_l_ft_v1  val_dataset.jsonl


In [4]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community qdrant_client langchain_qdrant sentence_transformers fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 

In [5]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.


In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [8]:
CONTEXTUAL_CHUNKS_FOLDER="/content/drive/MyDrive/MgAlloy_Finetune_Dataset/contextual_chunks"
COLLECTIONS_PATH="/content/drive/MyDrive/MgAlloy_Finetune_Dataset/collections"

## Load Test Data

In [39]:
import pickle
import tqdm
from tqdm import tqdm
from langchain_core.documents import Document

def load_chunks_as_documents(path):

    file_list = []
    if os.path.isfile(path) and path.endswith('.pkl'):
        # Single pkl file
        file_list.append(path)
    elif os.path.isdir(path):
        # Directory of pkl files
        for filename in os.listdir(path):
            if filename.endswith('.pkl'):
                path_ = os.path.join(path, filename)
                file_list.append(path_)

    loaded_chunk_data = {}
    for file in file_list:
        with open(file, 'rb') as f:
            data = pickle.load(f)
            loaded_chunk_data[data["filename"]] = data["chunks"]

    print(f"Loaded {len(loaded_chunk_data)} documents from {path}:")
    for i, doc_name in enumerate(loaded_chunk_data.keys()):
        print(f"  {i+1}. {doc_name}")

    # Convert the chunks to langhcain documents
    documents = []
    for fname in tqdm(loaded_chunk_data.keys(), desc="Converting chunks to documents"):

        chunks = loaded_chunk_data[fname]
        for chunk in chunks:
            documents.append(
                Document(
                    page_content=chunk.page_content,
                    metadata=chunk.metadata
                )
            )
    return documents


In [40]:
training_documents = load_chunks_as_documents(CONTEXTUAL_CHUNKS_FOLDER)

Loaded 6 documents from /content/drive/MyDrive/MgAlloy_Finetune_Dataset/contextual_chunks:
  1. Fatigue_behaviour_and_fractography_of_extruded_AZ80
  2. Characterization_of_forged_magnesium_alloys
  3. Application_of_Machine_Learning_Modeling_in_Establishing_the_Process_Structure_and_Property_Relationships_of_the_Cast-Forged_AZ80_Magnesium_Alloy
  4. A_Method_for_Comparing_the_Fatigue_Performance_of_Forged
  5. Microstructure_and_texture_evolution_during_hot_deformation_of_AZ80_magnesium_alloy
  6. Fatigue_of_Forged_AZ80_Magnesium_Alloy


Converting chunks to documents: 100%|██████████| 6/6 [00:00<00:00, 2058.22it/s]


In [41]:
print(f"Number of documents {len(training_documents)}")

Number of documents 638


In [ ]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [ ]:
training_split_documents = training_documents[:len(training_documents) - 192]
val_split_documents = training_documents[len(training_documents) - 192:638-94]
test_split_documents = training_documents[638-94:]

In [ ]:
print(f"Number of training documents {len(training_split_documents)}")
print(f"Number of validation documents {len(val_split_documents)}")
print(f"Number of testing documents {len(test_split_documents)}")

Number of training documents 446
Number of validation documents 98
Number of testing documents 94


## Constructing a Fine-tuning Dataset

In [14]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [ ]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [ ]:
import tqdm

async def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  ### YOUR CODE HERE
  id_set = set()
  for document in tqdm.tqdm(documents, desc="Processing documents"):
    # Generate question IDs and questions for each document
    for _ in range(n_questions):
      question_id = str(uuid.uuid4())
      while question_id in id_set:
        question_id = str(uuid.uuid4())
      id_set.add(question_id)

      # Generate a question based on document content
      questions[question_id] = await question_generation_chain.ainvoke({"context": document.page_content, "n_questions": n_questions})

      # Associate question with document ID
      relevant_docs[question_id] = [document.metadata["id"]]

  return questions, relevant_docs

In [ ]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

Processing documents: 100%|██████████| 446/446 [22:45<00:00,  3.06s/it]


In [ ]:
import json
# save datasets to gdrive
dataset_path = "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

In [ ]:
training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

training_questions = {k: str(v) for k, v in training_questions.items()}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open(os.path.join(dataset_path,"training_dataset.jsonl"), "w") as f:
  json.dump(train_dataset, f)

In [ ]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)

Processing documents: 100%|██████████| 98/98 [06:05<00:00,  3.73s/it]


In [ ]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_questions = {k: str(v) for k, v in val_questions.items()}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open(os.path.join(dataset_path,"val_dataset.jsonl"), "w") as f:
  json.dump(val_dataset, f)

In [ ]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Processing documents: 100%|██████████| 94/94 [06:34<00:00,  4.20s/it]


In [ ]:
test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_questions = {k: str(v) for k, v in test_questions.items()}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : test_corpus
}

with open(os.path.join(dataset_path,"test_dataset.jsonl"), "w") as f:
  json.dump(test_dataset, f)

## Fine-tuning snowflake-arctic-embed-l

In [ ]:
# load ft datasets
import os
import json
dataset_path = "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

with open(os.path.join(dataset_path,"training_dataset.jsonl"), "r") as f:
    train_dataset = json.load(f)

with open(os.path.join(dataset_path,"test_dataset.jsonl"), "r") as f:
    test_dataset = json.load(f)

with open(os.path.join(dataset_path,"val_dataset.jsonl"), "r") as f:
    val_dataset = json.load(f)

In [ ]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [ ]:
BATCH_SIZE = 16

In [ ]:
# move dataset into the expected format for training.
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [ ]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
#!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

In [ ]:
#!pip install sentence-transformers==3.3.1

In [ ]:
#!pip install datasets==3.3.0

In [ ]:
EPOCHS = 25

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

''' warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path=os.path.join(dataset_path, 'mg_alloy_finetuned_arctic_embed_l_ft_v2'),
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
) '''

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.897959,0.979592,0.989796,0.989796,0.897959,0.326531,0.197959,0.098980,0.897959,0.979592,0.989796,0.989796,0.952522,0.939626,0.939751
56,No log,No log,0.928571,0.979592,0.989796,0.989796,0.928571,0.326531,0.197959,0.098980,0.928571,0.979592,0.989796,0.989796,0.963820,0.954932,0.955061
100,No log,No log,0.959184,0.989796,0.989796,1.000000,0.959184,0.329932,0.197959,0.100000,0.959184,0.989796,0.989796,1.000000,0.980111,0.973810,0.973810
112,No log,No log,0.943878,0.984694,0.989796,1.000000,0.943878,0.328231,0.197959,0.100000,0.943878,0.984694,0.989796,1.000000,0.974337,0.965914,0.965914
150,No log,No log,0.959184,1.000000,1.000000,1.000000,0.959184,0.333333,0.200000,0.100000,0.959184,1.000000,1.000000,1.000000,0.984936,0.979592,0.979592
168,No log,No log,0.948980,1.000000,1.000000,1.000000,0.948980,0.333333,0.200000,0.100000,0.948980,1.000000,1.000000,1.000000,0.981170,0.974490,0.974490
200,No log,No log,0.964286,1.000000,1.000000,1.000000,0.964286,0.333333,0.200000,0.100000,0.964286,1.000000,1.000000,1.000000,0.986819,0.982143,0.982143
224,No log,No log,0.989796,1.000000,1.000000,1.000000,0.989796,0.333333,0.200000,0.100000,0.989796,1.000000,1.000000,1.000000,0.996234,0.994898,0.994898
250,No log,No log,0.979592,1.000000,1.000000,1.000000,0.979592,0.333333,0.200000,0.100000,0.979592,1.000000,1.000000,1.000000,0.992468,0.989796,0.989796
280,No log,No log,0.984694,1.000000,1.000000,1.000000,0.984694,0.333333,0.200000,0.100000,0.984694,1.000000,1.000000,1.000000,0.994351,0.992347,0.992347


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hf_username = "tabesink92"
model.push_to_hub(f"{hf_username}/mg_alloy-snowflake-arctic-embed-l-ft-v2")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2/commit/ad03d4d13946948f4ff006256270f41be52fd664'

## Load Fine-tuning snowflake-arctic-embed-l

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/77.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Evaluating Retriever

In [26]:
!pip install faiss-cpu tqdm

In [27]:
# load ft datasets
import os
import json
dataset_path = "/content/drive/MyDrive/MgAlloy_Finetune_Dataset"

with open(os.path.join(dataset_path,"training_dataset.jsonl"), "r") as f:
    train_dataset = json.load(f)

with open(os.path.join(dataset_path,"test_dataset.jsonl"), "r") as f:
    test_dataset = json.load(f)

with open(os.path.join(dataset_path,"val_dataset.jsonl"), "r") as f:
    val_dataset = json.load(f)

In [28]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [32]:
from tqdm import tqdm
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

In [33]:
# Snowflake/snowflake-arctic-embed-l (base)
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 188/188 [00:05<00:00, 37.09it/s]


In [34]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [35]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.7978723404255319

In [37]:
# Snowflake/snowflake-arctic-embed-l (fine-tuned)
from sentence_transformers import SentenceTransformer
#finetune_embeddings = SentenceTransformer("tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2")

finetune_embeddings = HuggingFaceEmbeddings(model_name="tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)
finetune_results_df = pd.DataFrame(finetune_results)
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

Some weights of BertModel were not initialized from the model checkpoint at tabesink92/mg_alloy-snowflake-arctic-embed-l-ft-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 188/188 [00:05<00:00, 37.00it/s]


1.0

#### RAG - Base Chain

In [42]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

In [43]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

In [44]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [45]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

#### Fine-tuned Chain

In [46]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [47]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

### RAGAS Evaluation

In [48]:
!pip install -qU ragas==0.2.10 rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.4 MB/s eta 0:00:00


In [54]:
training_documents = training_documents[:25]

In [56]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(training_documents, testset_size=10)

dataset.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/25 [00:00<?, ?it/s]

ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node
ERROR:ragas.testset.transforms.engine:unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/33 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How does Mirzadeh's kinetic modeling approach ...,[# Characterization of forged magnesium alloys...,Mirzadeh's kinetic modeling approach is suitab...,single_hop_specifc_query_synthesizer
1,What DRX do?,[2.4.2 Application of processing maps Hot defo...,DRX is desired because it is associated with m...,single_hop_specifc_query_synthesizer
2,What are the characteristics and processing co...,[Characterization of forged magnesium alloys T...,The AZ31 magnesium alloy contains the β-Mg17Al...,single_hop_specifc_query_synthesizer
3,Who Benoit MJ?,"[Selected results |(a)|As Cast|(b)|450 °C, 5 h...",Benoit MJ is one of the authors of the study o...,single_hop_specifc_query_synthesizer
4,How does the homogenization process affect the...,[<1-hop>\n\nCharacterization of forged magnesi...,The homogenization process significantly impac...,multi_hop_abstract_query_synthesizer
5,How do processing maps help in understanding t...,[<1-hop>\n\n2.4.2 Application of processing ma...,Processing maps are crucial in understanding t...,multi_hop_abstract_query_synthesizer
6,How does the dynamic materials model (DMM) add...,[<1-hop>\n\n# Characterization of forged magne...,The dynamic materials model (DMM) addresses th...,multi_hop_abstract_query_synthesizer
7,How does the dynamic materials model (DMM) add...,[<1-hop>\n\n# Characterization of forged magne...,The dynamic materials model (DMM) addresses th...,multi_hop_abstract_query_synthesizer
8,What role does Hadadzadeh play in the constitu...,[<1-hop>\n\n# Characterization of forged magne...,Hadadzadeh is involved in the constitutive mod...,multi_hop_specific_query_synthesizer
9,How do the dynamic materials model introduced ...,[<1-hop>\n\n2.4 Processing maps # 2.4.1 Concep...,The dynamic materials model (DMM) introduced b...,multi_hop_specific_query_synthesizer


In [59]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.3 MB/s eta 0:00:00


In [60]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


class State(TypedDict):
  question: str
  context: List[Document]
  response: str

def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

def retrieve(state):
  #retrieved_docs = retriever.invoke(state["question"])
  retrieved_docs = base_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [61]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [62]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [63]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,How does Mirzadeh's kinetic modeling approach ...,"[# References\n\n1. S. W. Xu, N. Matsumoto, S....",[# Characterization of forged magnesium alloys...,"H. Mirzadeh's kinetic modeling approach, as de...",Mirzadeh's kinetic modeling approach is suitab...,single_hop_specifc_query_synthesizer
1,What DRX do?,[# Table of Contents\n\n- List of Figures .......,[2.4.2 Application of processing maps Hot defo...,The provided context does not explicitly state...,DRX is desired because it is associated with m...,single_hop_specifc_query_synthesizer
2,What are the characteristics and processing co...,"[# References\n\n1. T. W. Wong, A. Hadadzadeh ...",[Characterization of forged magnesium alloys T...,The AZ31 magnesium alloy has several key chara...,The AZ31 magnesium alloy contains the β-Mg17Al...,single_hop_specifc_query_synthesizer
3,Who Benoit MJ?,[# Dedication\n\nTo the Guru .. who guides us\...,"[Selected results |(a)|As Cast|(b)|450 °C, 5 h...",The provided context does not contain any info...,Benoit MJ is one of the authors of the study o...,single_hop_specifc_query_synthesizer
4,How does the homogenization process affect the...,[# 9.3.2. Microstructure to mechanical propert...,[<1-hop>\n\nCharacterization of forged magnesi...,The provided context does not offer specific d...,The homogenization process significantly impac...,multi_hop_abstract_query_synthesizer
5,How do processing maps help in understanding t...,[microstructures. The results for all 11 cast-...,[<1-hop>\n\n2.4.2 Application of processing ma...,Processing maps are valuable tools in understa...,Processing maps are crucial in understanding t...,multi_hop_abstract_query_synthesizer
6,How does the dynamic materials model (DMM) add...,[# 9.3.2. Microstructure to mechanical propert...,[<1-hop>\n\n# Characterization of forged magne...,The dynamic materials model (DMM) addresses th...,The dynamic materials model (DMM) addresses th...,multi_hop_abstract_query_synthesizer
7,How does the dynamic materials model (DMM) add...,[# 9.3.2. Microstructure to mechanical propert...,[<1-hop>\n\n# Characterization of forged magne...,The dynamic materials model (DMM) addresses th...,The dynamic materials model (DMM) addresses th...,multi_hop_abstract_query_synthesizer
8,What role does Hadadzadeh play in the constitu...,[# 9.3.2. Microstructure to mechanical propert...,[<1-hop>\n\n# Characterization of forged magne...,"In the context provided, Hadadzadeh contribute...",Hadadzadeh is involved in the constitutive mod...,multi_hop_specific_query_synthesizer
9,How do the dynamic materials model introduced ...,[# 9.3.2. Microstructure to mechanical propert...,[<1-hop>\n\n2.4 Processing maps # 2.4.1 Concep...,The dynamic materials model introduced by Pras...,The dynamic materials model (DMM) introduced b...,multi_hop_specific_query_synthesizer


### Base Retriever

In [64]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [65]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[46]: LLMDidNotFinishException(The LLM generation was not completed. Please increase try increasing the max_tokens and try again.)
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()


{'context_recall': 0.0375, 'faithfulness': 0.3760, 'factual_correctness': 0.2842, 'answer_relevancy': 0.7266, 'context_entity_recall': 0.0038, 'noise_sensitivity_relevant': 0.0071}

### FINE_TUNED RETRIEVER

In [66]:
def retrieve(state):
  #retrieved_docs = retriever.invoke(state["question"])
  retrieved_docs = finetune_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [67]:
evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[28]: LLMDidNotFinishException(The LLM generation was not completed. Please increase try increasing the max_tokens and try again.)
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()


{'context_recall': 0.0208, 'faithfulness': 0.3642, 'factual_correctness': 0.3292, 'answer_relevancy': 0.7269, 'context_entity_recall': 0.0101, 'noise_sensitivity_relevant': 0.0371}